# High performance pandas

---

```py
mask = (x > 0.5) & (y < 0.5)
df[df[mask]]
```

under the hood

```py
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2
```

use pd.eval("") & pd.query() -> does elementwise operation with numexpr

In [2]:
import numpy as np
import pandas as pd

nrows, ncols = 1_000_000, 100
df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.593723,0.174748,0.671811,1.618193,0.501961,-0.429331,0.884543,0.297428,-0.243415,1.438506,...,-0.526547,0.491524,0.002904,0.559370,-1.434391,0.071990,0.824245,-1.020230,0.111951,-2.167939
1,-2.153320,2.603937,1.283022,-0.392131,1.653799,0.492181,0.612560,-1.196638,0.263258,1.445815,...,0.187566,0.180361,-0.032213,-1.100533,-0.213058,-0.600118,0.491017,-1.853190,0.864110,-1.107817
2,-0.563294,0.398744,-0.335973,0.432850,1.400227,-0.967751,0.042959,-1.044998,-0.614138,1.094560,...,-0.508020,0.384428,0.116048,-1.281179,-2.129831,0.508188,-2.317395,-0.700994,-0.421250,-1.425655
3,-2.004752,2.527116,0.050374,-1.086200,1.640246,-2.901131,-0.891809,-0.769053,1.777771,0.436954,...,-0.705056,-0.232664,-0.992507,0.048036,0.987572,-0.816902,-0.609405,0.046726,-0.122636,0.828489
4,0.986309,-0.790284,0.644957,-0.366543,0.023015,-0.635848,-0.614730,0.139058,-0.354178,-1.008683,...,0.359964,-0.593112,1.482242,0.669330,-1.470158,1.393788,-1.365512,-0.444817,0.854109,0.795505


In [4]:
df1.shape

(1000000, 100)

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [8]:
%timeit df1+df2+df3+df4
%timeit pd.eval("df1+df2+df3+df4")

620 ms ± 4.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
224 ms ± 6.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [15]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns= ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,5,1,1
1,3,3,4
2,2,3,1
3,5,3,4
4,3,3,4
5,3,5,4


In [18]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace= True)
rolls

,Die1,Die2,Die3,Sum
0,5,1,1,7
1,3,3,4,10
2,2,3,1,6
3,5,3,4,12
4,3,3,4,10
5,3,5,4,12


In [19]:
high = 11 
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,5,1,1,7,False
1,3,3,4,10,False
2,2,3,1,6,False
3,5,3,4,12,True
4,3,3,4,10,False
5,3,5,4,12,True


In [20]:
# traditional way to filter from a dataframe 
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum
3,5,3,4,12
5,3,5,4,12


# Query

- filter using query

In [22]:
rolls.query("Sum > @high & Die1 == 3")

,Die1,Die2,Die3,Sum
5,3,5,4,12


In [24]:
df_os = pd.read_csv("../projects/Data/athlete_events.csv")
df_os

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [26]:
df_os[df_os["NOC"] == "SWE"].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [27]:
df_os.query("NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [28]:
%timeit df_os[df_os["NOC"] == "SWE"].head()
%timeit df_os.query("NOC == 'SWE'").head()

15.2 ms ± 953 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.66 ms ± 353 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%timeit df_os[df_os["Height"] > 180]
%timeit df_os.query("Height > 180")

6.39 ms ± 28 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.61 ms ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [33]:
%timeit df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")]
%timeit df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

25.8 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.73 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
